In [1]:
import pandas as pd
import os
import gc
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler
from gensim.models import Word2Vec
import math
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.columns

Index(['客户ID', '地理区域', '是否双频', '是否翻新机', '当前手机价格', '手机网络功能', '婚姻状况', '家庭成人人数',
       '信息库匹配', '预计收入', '信用卡指示器', '当前设备使用天数', '在职总月数', '家庭中唯一订阅者的数量',
       '家庭活跃用户数', '新手机用户', '信用等级代码', '平均月费用', '每月平均使用分钟数', '平均超额使用分钟数',
       '平均超额费用', '平均语音费用', '数据超载的平均费用', '平均漫游呼叫数', '当月使用分钟数与前三个月平均值的百分比变化',
       '当月费用与前三个月平均值的百分比变化', '平均掉线语音呼叫数', '平均丢弃数据呼叫数', '平均占线语音呼叫数',
       '平均占线数据调用次数', '平均未接语音呼叫数', '未应答数据呼叫的平均次数', '尝试拨打的平均语音呼叫次数',
       '尝试数据调用的平均数', '平均接听语音电话数', '平均完成的语音呼叫数', '完成数据调用的平均数', '平均客户服务电话次数',
       '使用客户服务电话的平均分钟数', '一分钟内的平均呼入电话数', '平均三通电话数', '已完成语音通话的平均使用分钟数',
       '平均呼入和呼出高峰语音呼叫数', '平均峰值数据调用次数', '使用高峰语音通话的平均不完整分钟数', '平均非高峰语音呼叫数',
       '非高峰数据呼叫的平均数量', '平均掉线或占线呼叫数', '平均尝试调用次数', '平均已完成呼叫数', '平均呼叫转移呼叫数',
       '平均呼叫等待呼叫数', '账户消费限额', '客户生命周期内的总通话次数', '客户生命周期内的总使用分钟数', '客户生命周期内的总费用',
       '计费调整后的总费用', '计费调整后的总分钟数', '计费调整后的呼叫总数', '客户生命周期内平均月费用',
       '客户生命周期内的平均每月使用分钟数', '客户整个生命周期内的平均每月通话次数', '过去三个月的平均每月使用分钟数',
       '过去三个月的平均每月通话次数', '过去三个月的平均月费用', '过去六个月的平均每月使用分钟数', '过

In [4]:
data = pd.concat([train, test], axis=0, ignore_index=True)

## 训练数据/测试数据准备

In [5]:
features = [f for f in data.columns if f not in ['是否流失','客户ID']]

train = data[data['是否流失'].notnull()].reset_index(drop=True)
test = data[data['是否流失'].isnull()].reset_index(drop=True)

x_train = train[features]
x_test = test[features]

y_train = train['是否流失']

## 模型训练
- 直接构建了一个函数，可以调用三种树模型，方便快捷

In [6]:
def cv_model(clf, train_x, train_y, test_x, clf_name):
    folds = 5
    seed = 2022
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 5,
                'num_leaves': 2 ** 5,
                'lambda_l2': 10,
                'feature_fraction': 0.7,
                'bagging_fraction': 0.7,
                'bagging_freq': 10,
                'learning_rate': 0.2,
                'seed': 2022,
                'n_jobs':-1
            }

            model = clf.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], 
                              categorical_feature=[], verbose_eval=3000, early_stopping_rounds=200)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
            
            print(list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])
                
        if clf_name == "xgb":
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            test_matrix = clf.DMatrix(test_x)
            
            params = {'booster': 'gbtree',
                      'objective': 'binary:logistic',
                      'eval_metric': 'auc',
                      'gamma': 1,
                      'min_child_weight': 1.5,
                      'max_depth': 5,
                      'lambda': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'eta': 0.2,
                      'tree_method': 'exact',
                      'seed': 2020,
                      'nthread': 36,
                      "silent": True,
                      }
            
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
            
            model = clf.train(params, train_matrix, num_boost_round=50000, evals=watchlist, verbose_eval=3000, early_stopping_rounds=200)
            val_pred  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
            test_pred = model.predict(test_matrix , ntree_limit=model.best_ntree_limit)
                 
        if clf_name == "cat":
            params = {'learning_rate': 0.2, 'depth': 5, 'l2_leaf_reg': 10, 'bootstrap_type': 'Bernoulli',
                      'od_type': 'Iter', 'od_wait': 50, 'random_seed': 11, 'allow_writing_files': False}
            
            model = clf(iterations=20000, **params)
            model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                      cat_features=[], use_best_model=True, verbose=3000)
            
            val_pred  = model.predict(val_x)
            test_pred = model.predict(test_x)
            
        train[valid_index] = val_pred
        test = test_pred / kf.n_splits
        cv_scores.append(roc_auc_score(val_y, val_pred))
        
        print(cv_scores)
       
    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    return train, test

In [7]:
def lgb_model(x_train, y_train, x_test):
    lgb_train, lgb_test = cv_model(lgb, x_train, y_train, x_test, "lgb")
    return lgb_train, lgb_test

def xgb_model(x_train, y_train, x_test):
    xgb_train, xgb_test = cv_model(xgb, x_train, y_train, x_test, "xgb")
    return xgb_train, xgb_test

def cat_model(x_train, y_train, x_test):
    cat_train, cat_test = cv_model(CatBoostRegressor, x_train, y_train, x_test, "cat") 
    return cat_train, cat_test

In [8]:
lgb_train, lgb_test = lgb_model(x_train, y_train, x_test)


************************************ 1 ************************************
[LightGBM] [Info] Number of positive: 60072, number of negative: 59928
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009464 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10515
[LightGBM] [Info] Number of data points in the train set: 120000, number of used features: 67
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500600 -> initscore=0.002400
[LightGBM] [Info] Start training from score 0.002400
Training until validation scores don't improve for 200 rounds
[3000]	training's auc: 0.999516	valid_1's auc: 0.812108
[6000]	training's auc: 1	valid_1's auc: 0.832951
[9000]	training's auc: 1	valid_1's auc: 0.840899
Early stopping, best iteration is:
[11399]	training's auc: 1	valid_1's auc: 0.843936
[('当前设备使用天数', 23206.93494541943), ('当月使用分钟数与前三个月平均值的百分比变化', 18

In [13]:
xgb_train, xgb_test = xgb_model(x_train, y_train, x_test)


************************************ 1 ************************************
[19:57:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.62651	eval-auc:0.61938
[3000]	train-auc:0.99330	eval-auc:0.79046
[6000]	train-auc:1.00033	eval-auc:0.80912
[9000]	train-auc:1.00043	eval-auc:0.81392
[10249]	train-auc:1.00043	eval-auc:0.81494
[0.8150796492074857]
************************************ 2 ************************************
[20:15:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters ar

In [14]:
cat_train, cat_test = cat_model(x_train, y_train, x_test)


************************************ 1 ************************************
0:	learn: 0.4955489	test: 0.4954619	best: 0.4954619 (0)	total: 104ms	remaining: 34m 45s
3000:	learn: 0.3769726	test: 0.4483572	best: 0.4483572 (3000)	total: 25.1s	remaining: 2m 22s
6000:	learn: 0.3209359	test: 0.4391546	best: 0.4391520 (5999)	total: 53.4s	remaining: 2m 4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.4360869428
bestIteration = 7499

Shrink model to first 7500 iterations.
[0.78868229695141]
************************************ 2 ************************************
0:	learn: 0.4953117	test: 0.4954092	best: 0.4954092 (0)	total: 15.4ms	remaining: 5m 8s
3000:	learn: 0.3763302	test: 0.4490481	best: 0.4490378 (2981)	total: 25.6s	remaining: 2m 24s
6000:	learn: 0.3196365	test: 0.4402621	best: 0.4402621 (6000)	total: 52s	remaining: 2m 1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.4361341716
bestIteration = 8001

Shrink model to first 8002 iterations.
[0.788

In [15]:
test['是否流失'] = lgb_test

In [16]:
test[['客户ID','是否流失']].to_csv('test_sub.csv', index=False)